In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy


import numpy as np
from sklearn.manifold import TSNE


Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [38]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [39]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')

In [40]:
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [41]:
top200 =['spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'q25_roll_std_100',
 '3th_peak_freq',
 'min_roll_std_100',
 'q05_roll_std_100',
 'iqr_6',
 'abs_max_8',
 'mfcc_5_abs_mean',
 "number_peaks{'n': 10}",
 'ave10_7',
 'mfcc_13_mean',
 "autocorrelation{'lag': 5}",
 'mfcc_accelerate_8_variance',
 'abs_max_7',
 'mfcc_accelerate_1_kurtosis',
 'q05_roll_std_1000',
 'spkt_welch_density__coeff_42',
 'max_to_min_diff_5',
 'mfcc_13_quantile25',
 'fft_coefficientcoeff_80__attr_"imag"',
 'abs_q25_5',
 'mfcc_5_mean',
 'median__roll_std',
 '5000skewness_max_',
 'fft_coefficientcoeff_6__attr_"abs"',
 'partial_autocorrelationlag_5',
 'abs_min_8',
 'spkt_welch_density__coeff_28',
 'ar_coefficientk_10__coeff_3',
 'abs_q75_7',
 'mfcc_accelerate_15_min',
 'abs_max_4',
 'mfcc_10_quantile25',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_density__coeff_79',
 "value_count{'value': 1}",
 '5000peak_peak_amp_max_',
 'min__roll_std',
 'mfcc_4_rolling_std_mean',
 'q01_roll_std_1000',
 'mfcc_3_abs_q75',
 'mfcc_5_quantile95',
 'fft_coefficientcoeff_16__attr_"imag"',
 'abs_q01_4',
 'mfcc_6_quantile05',
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 '5000quantile25skewness_',
 'mfcc_0_abs_q25',
 'mfcc_delta_3_quantile01',
 'spkt_welch_density__coeff_99',
 'mfcc_9_mean',
 'mfcc_12_mean',
 'q75_roll_mean_10',
 'max_to_min',
 'mfcc_3_abs_mean',
 'kurt_7',
 'mfcc_14_quantile01',
 'mfcc_10_mean',
 'mfcc_delta_13_min',
 'mfcc_delta_13_abs_max',
 '5000no_zero_crossing_mean_',
 'abs_q75_6',
 'mfcc_12_quantile99',
 'mfcc_accelerate_6_skewness',
 'mfcc_5_max',
 '5000smoothness_entropy_',
 '5000median_skewness_',
 'mfcc_accelerate_8_min',
 'abs_min_3',
 '5000quantile75mean_',
 "number_crossing_m{'m': 1}",
 '5000smoothness_std_',
 'mfcc_9_abs_q25',
 'flac3_1_quantile99',
 'mfcc_5_skewness',
 'fft_coefficientcoeff_56__attr_"angle"',
 'fft_coefficientcoeff_70__attr_"abs"',
 'mfcc_10_abs_q75',
 'fft_coefficientcoeff_24__attr_"angle"',
 'med_7',
 'spkt_welch_density__coeff_73',
 'abs_q99_8',
 'ave10_6',
 'spkt_welch_density__coeff_38',
 'skew_1',
 'mfcc_delta_3_abs_q95',
 "change_quantiles{'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}",
 'max_to_min_5',
 'mfcc_delta_4_quantile75',
 'abs_max_1',
 '5000crest_factor_quantile75',
 'partial_autocorrelationlag_1',
 'mfcc_delta_11_quantile01',
 'mfcc_accelerate_7_quantile01',
 'mfcc_8_rolling_std_mean',
 'q95_roll_mean_100',
 'mfcc_delta_6_rolling_std_mean',
 'flac3_0_min',
 'fft_coefficientcoeff_36__attr_"abs"',
 'iqr_8',
 'max_9',
 '5000smoothness_quantile05',
 'mfcc_1_kurtosis',
 'mfcc_7_abs_q95',
 'spkt_welch_density__coeff_66',
 'spkt_welch_density__coeff_64',
 'mfcc_3_abs_q95',
 '5000skewness_mean_',
 'mfcc_10_abs_mean',
 '5000quantile99quantile01',
 'mean_change_rate',
 'mfcc_accelerate_5_rolling_std_mean',
 'mfcc_accelerate_5_quantile05',
 '5000rms_median_',
 'flac3_1_abs_mean',
 '5000rms_quantile05',
 '5000quantile75quantile25',
 'mfcc_accelerate_12_min',
 "value_count{'value': -1}",
 'mfcc_12_abs_max',
 'abs_min_5',
 'mfcc_5_abs_q99',
 'mfcc_delta_5_quantile01',
 'mfcc_12_quantile75',
 'fft_coefficientcoeff_56__attr_"imag"',
 'spkt_welch_densitycoeff_5',
 'mfcc_0_abs_q05',
 'mfcc_13_quantile75',
 'mfcc_delta_5_min',
 'mfcc_5_quantile99',
 'fft_coefficientcoeff_8__attr_"angle"',
 'spkt_welch_density__coeff_30',
 'mfcc_accelerate_4_max',
 'mfcc_14_median',
 "change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'}",
 'mfcc_5_median',
 'mfcc_accelerate_5_abs_q75',
 'spkt_welch_density__coeff_59',
 'mfcc_accelerate_5_kurtosis',
 'mfcc_delta_9_quantile99',
 'mfcc_5_quantile25',
 'spkt_welch_density__coeff_113',
 'mfcc_2_quantile05',
 'spkt_welch_density__coeff_58',
 'mfcc_5_abs_q75',
 'spkt_welch_density__coeff_22',
 'spkt_welch_density__coeff_115',
 'spkt_welch_density__coeff_4',
 'mfcc_5_abs_q05',
 'spkt_welch_density__coeff_25',
 'mfcc_5_abs_max',
 'spkt_welch_density__coeff_27',
 'mfcc_4_quantile75',
 'mfcc_3_rolling_std_mean',
 'mfcc_4_median',
 'mfcc_0_quantile01',
 '5000quantile75rssq_',
 'fft_coefficientcoeff_24__attr_"imag"',
 'mfcc_delta_12_kurtosis',
 '5000rms_quantile25',
 'mfcc_13_quantile05',
 'fft_coefficientcoeff_8__attr_"imag"',
 'mfcc_delta_11_rolling_std_mean',
 'mfcc_delta_12_quantile99',
 'mfcc_accelerate_9_abs_q25',
 'mfcc_11_abs_mean',
 'mfcc_delta_15_abs_max',
 'abs_max_2',
 'mfcc_11_abs_std',
 'abs_max_roll_mean_1000',
 'peak_to_average_power_ratio__roll_mean',
 'mfcc_13_median',
 'fft_coefficientcoeff_62__attr_"abs"',
 'mfcc_11_quantile75',
 "quantile{'q': 0.8}",
 'mfcc_delta_3_rolling_std_mean',
 'abs_q99_7',
 'mfcc_delta_12_max',
 'mfcc_12_min',
 'mfcc_10_quantile99',
 'mfcc_14_abs_q99',
 'mfcc_14_quantile05',
 'mfcc_14_quantile25',
 'kurt_1',
 'mfcc_2_median',
 'q01_2',
 'mfcc_2_abs_q25',
 'kurt_8',
 'mfcc_accelerate_6_abs_max',
 'mfcc_1_quantile75',
 'q05_5',
 'abs_q95_9',
 'q05_roll_mean_100',
 'mfcc_10_abs_q25',
 'q75_9',
 'mfcc_15_mean',
 "change_quantiles{'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'}",
 'flac3_0_abs_std',
 "change_quantiles{'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}"]

In [46]:
group = EP.bubble_merge(df_train, top200,)

In [52]:
def pca_com(df_train, df_test, group):
    
    df_train_pca = pd.DataFrame()
    df_test_pca = pd.DataFrame()
    df_train_pca['index'] = copy.deepcopy(df_train['index'])
    df_test_pca['index'] = copy.deepcopy(df_test['index'])

    for k,v in group.items():
        
        if len(v) > 1:
        
            X_train = df_train[v].values
            X_test = df_test[v].values
            scaler = StandardScaler()
            X_train_norm = scaler.fit_transform(X_train)
            X_test_norm = scaler.transform(X_test)
            pca = PCA(n_components=len(v))
            pca.fit(X_train_norm)
            X_com_train = pca.transform(X_train_norm)
            X_com_test = pca.transform(X_test_norm)

            for i in range(len(v)):
                df_train_pca['{}_pca{}'.format(k, i)] = X_com_train[:,i]
                df_test_pca['{}_pca{}'.format(k, i)] = X_com_test[:,i]
    
    return df_train_pca, df_test_pca
    
    

In [53]:
df_train_pca, df_test_pca = pca_com(df_train, df_test, group)

In [54]:
df_train_pca.tail()

,index,spkt_welch_densitycoeff_2_pca0,spkt_welch_densitycoeff_2_pca1,spkt_welch_densitycoeff_2_pca2,spkt_welch_densitycoeff_2_pca3,spkt_welch_densitycoeff_2_pca4,spkt_welch_densitycoeff_2_pca5,spkt_welch_densitycoeff_2_pca6,q25_roll_std_100_pca0,q25_roll_std_100_pca1,...,mfcc_2_median_pca2,q05_5_pca0,q05_5_pca1,q05_5_pca2,q05_5_pca3,q05_5_pca4,q05_5_pca5,q05_5_pca6,q05_5_pca7,q05_5_pca8
33421,628904423,-0.279066,-0.005498,0.126937,-0.064018,0.001328,0.000890,0.009237,-4.829230,0.476553,...,-0.085084,-2.798112,0.703290,-0.014292,0.044419,0.120970,0.178078,0.013097,0.032446,0.020596
33422,628923173,-0.299300,-0.000660,0.156584,-0.096718,0.008344,-0.014920,0.009037,-5.089634,0.619544,...,-0.069171,-2.882967,0.698219,-0.038024,-0.014568,0.090545,0.190152,-0.049540,-0.017506,0.030519
33423,628941923,-0.298948,-0.000359,0.156478,-0.096975,0.008667,-0.014411,0.009051,-5.039301,0.681081,...,0.093539,-2.891229,0.538484,-0.089359,0.115928,0.084229,0.125249,0.019906,-0.045645,0.006263
33424,628960673,-0.295932,-0.007883,0.148668,-0.073140,0.000725,0.009057,0.012582,-5.163416,0.758344,...,-0.036686,-2.915321,0.568581,-0.076947,0.105861,0.081604,0.142069,-0.007726,-0.031183,0.007389
33425,628979423,-0.295580,-0.007528,0.148548,-0.073413,0.001062,0.009595,0.012617,-4.932492,0.729456,...,-0.043040,-2.713628,0.612797,-0.134072,-0.054111,0.069982,0.127138,0.034240,-0.075580,0.002804


In [55]:
df_test_pca.tail()

,index,spkt_welch_densitycoeff_2_pca0,spkt_welch_densitycoeff_2_pca1,spkt_welch_densitycoeff_2_pca2,spkt_welch_densitycoeff_2_pca3,spkt_welch_densitycoeff_2_pca4,spkt_welch_densitycoeff_2_pca5,spkt_welch_densitycoeff_2_pca6,q25_roll_std_100_pca0,q25_roll_std_100_pca1,...,mfcc_2_median_pca2,q05_5_pca0,q05_5_pca1,q05_5_pca2,q05_5_pca3,q05_5_pca4,q05_5_pca5,q05_5_pca6,q05_5_pca7,q05_5_pca8
2619,seg_ff4236,-0.303887,-0.019357,0.163520,-0.050491,-0.009518,0.076967,0.025570,0.893577,-0.374148,...,0.037285,0.103995,-0.148570,-0.091759,-0.014504,0.245148,-0.115819,0.032623,-0.035615,-0.085929
2620,seg_ff7478,-0.000669,-0.045994,-0.175282,0.279016,-0.051730,0.174285,0.015218,-0.927250,-0.139274,...,-0.196292,-0.886536,-0.177688,-0.141848,0.178253,0.056539,-0.123452,0.051731,-0.100225,-0.006325
2621,seg_ff79d9,-0.148414,0.016660,-0.002848,0.020948,0.008120,0.106044,0.008289,3.552705,-1.088915,...,-0.176199,1.924337,-0.357671,-0.911054,-0.384203,0.111543,-0.158670,0.035644,0.154566,0.011697
2622,seg_ffbd6a,0.097202,-0.015996,-0.253687,0.186921,-0.035866,0.058759,0.007145,6.458916,-0.427098,...,0.182617,4.680188,-0.153080,-0.805584,-0.297494,0.619454,0.227968,0.230371,0.327926,-0.098604
2623,seg_ffe7cc,-0.332063,-0.020380,0.195769,-0.075890,-0.007612,0.060031,0.026040,-4.634889,0.460507,...,-0.180384,-2.668025,0.577427,-0.424513,-0.238045,0.169196,0.065255,-0.176035,0.033339,0.000663


In [57]:
df_train_pca.to_pickle('../feats/pca2_train.pkl')

In [58]:
df_test_pca.to_pickle('../feats/pca2_test.pkl')